# Preâmbulo do trabalho
Carregando a instalando pacotes necessários

In [52]:
import pandas as pd
!pip install faker
from faker import Faker
from random import randint
from random import sample

# Funções de criação do banco de dados
Aqui são geradas as funções que, posteriormente, serão usadas para criar os diferentes bancos de dados.

In [53]:
fake = Faker("pt_BR")

def vend(x):
    # pandas dataframe
    data = pd.DataFrame()
    data['id'] = sample(range(10),x)
    for i in range(0, x):
        data.loc[i,'nome'] = fake.first_name()
    return data

def empresa(x):
    # pandas dataframe
    data = pd.DataFrame()
    data['id'] = sample(range(15),x)
    for i in range(0, x):
        data.loc[i,'empresa']= fake.company()
    return data

def dados(x):
    # pandas dataframe
    data = pd.DataFrame()
    for i in range(0, x):
        data.loc[i,'id_vend']= sample(range(10),1)[0]
        data.loc[i,'id_empr']= sample(range(15),1)[0]
        data.loc[i,'estado']= fake.estado_sigla()
        data.loc[i,'vendas']= sample(range(2,125),1)[0]
        data.loc[i,'valor']= data.loc[i,'vendas'] * randint(5,20)
    return data

# Geração do banco de dados para as análises
Importante deixar claro que apenas o comando *Faker.seed* deve ser modificado com o número do cartão. Os demais valores seguem os mesmos.

In [54]:
Faker.seed(577787)  ## substituir o valor entre parentêses pelo número do cartão

In [55]:
vendedores = vend(5);  ## codigo dos vendedores conhecidos
empresas = empresa(10);  ## codigo das empresas conhecidas
dados = dados(300);  ## banco de dados das vendas

In [56]:
# Abaixo segue o código do trabalho propriamente dito. Bom trabalho!


# Início do código de solução


Montando a amostra aleatória simples e adicionando a ela o nome da empresa e vendedor

In [57]:
dados_descritiva = dados.sample(frac = 0.4)

dados_descritiva = pd.merge(dados_descritiva, vendedores, how='left', left_on='id_vend', right_on='id')
dados_descritiva = dados_descritiva.drop('id', axis=1)
dados_descritiva['nome'].fillna('Desconhecido', inplace=True)

dados_descritiva = pd.merge(dados_descritiva, empresas, how='left', left_on='id_empr', right_on='id')
dados_descritiva = dados_descritiva.drop('id', axis=1)
dados_descritiva['empresa'].fillna('Não Informado', inplace=True)


Montando os "rankings" de acordo com as especificações apresentadas:

In [58]:
respostas = ['dados_descritiva']

comb = [('id_vend', 'vendas'),
        ('id_vend', 'valor'),
        ('id_empr', 'vendas'),
        ('id_empr', 'valor')]

for a, b in comb:

  temp_df_name = f'resum_{a}_{b}'
  temp_df = dados_descritiva.groupby(a)[b].sum().reset_index()

  if a == 'id_vend':
    temp_df = pd.merge(temp_df, vendedores, how='left', left_on=a, right_on='id')
    temp_df = temp_df.drop('id', axis=1)
    temp_df['nome'].fillna('Desconhecido', inplace=True)
  else:
    temp_df = pd.merge(temp_df, empresas, how='left', left_on=a, right_on='id')
    temp_df = temp_df.drop('id', axis=1)
    temp_df['empresa'].fillna('Não Informado', inplace=True)

  temp_df = temp_df.sort_values(by=b, ascending=False)

  locals()[temp_df_name] = temp_df
  respostas.append(temp_df_name)

Mostrando todos os dataframes montados anteriormente:

In [59]:
for i in respostas:
  j = locals()[i]
  print(j)
  print('##' * 20)

     id_vend  id_empr estado  vendas   valor          nome             empresa
0        9.0      8.0     RS    80.0  1440.0         Kevin       Não Informado
1        5.0      3.0     PR    14.0   252.0  Desconhecido             Peixoto
2        1.0      9.0     GO    48.0   432.0  Desconhecido       Não Informado
3        9.0     13.0     AP    88.0  1496.0         Kevin       da Rocha - ME
4        5.0     12.0     PB   117.0  2106.0  Desconhecido             Rezende
..       ...      ...    ...     ...     ...           ...                 ...
115      6.0      2.0     PB    32.0   224.0  Desconhecido       Não Informado
116      3.0      4.0     RO    75.0  1500.0         Maitê   Vieira Jesus - ME
117      7.0      1.0     BA    86.0  1720.0         Luana  Duarte Cardoso S/A
118      9.0      7.0     GO    72.0  1152.0         Kevin             Moreira
119      9.0     10.0     RN    88.0  1408.0         Kevin               Porto

[120 rows x 7 columns]
############################